# Face-recognition

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Définir le chemin d'accès aux images
folder_path = "C:\\Users\\MSI\\Downloads\\archive (5)"
train_folder = os.path.join("C:\\Users\\MSI\\Downloads\\archive (5)", "train")

test_folder = os.path.join("C:\\Users\\MSI\\Downloads\\archive (5)", 'test')

# Taille des images
picture_size = 48



In [ ]:
def load_images_from_folder(folder):
    images = []
    labels = []
    if not os.path.exists(folder):
        raise FileNotFoundError(f"Le dossier {folder} n'existe pas.")
    
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        
        # Vérifiez si c'est un dossier
        if not os.path.isdir(label_folder):
            print(f"Avertissement: {label_folder} n'est pas un dossier.")
            continue
        
        for img_file in os.listdir(label_folder):
            img_path = os.path.join(label_folder, img_file)
            print(f"Tentative de chargement de l'image : {img_path}")  # Affichage du chemin
            
            img = cv2.imread(img_path)
            
            # Vérifiez si l'image a été chargée avec succès
            if img is None:
                print(f"Avertissement: Impossible de charger l'image à partir du chemin: {img_path}")
                continue  # Passez à l'image suivante si l'image n'est pas valide
            
            img = cv2.resize(img, (picture_size, picture_size))
            img = img_to_array(img)
            images.append(img)
            labels.append(label)  # Ajoutez ici l'étiquette correspondante
    
    return np.array(images), np.array(labels)

# Charger les données d'entraînement et de test
X_train, y_train = load_images_from_folder(train_folder)

# Vérifiez si le dossier de test existe avant de charger
if not os.path.exists(test_folder):
    print(f"Le dossier de test {test_folder} n'existe pas. Veuillez vérifier la structure du dossier.")
else:
    X_test, y_test = load_images_from_folder(test_folder)


In [ ]:

# Charger les données d'entraînement et de test
X_train, y_train = load_images_from_folder(train_folder)
X_test, y_test = load_images_from_folder(test_folder)

# Normaliser les données
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
# Convertir les étiquettes en catégories
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [ ]:
# Création du modèle CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(picture_size, picture_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(y_train))))
model.add(Activation('softmax'))

# Compiler le modèle
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
# Entraîner le modèle
model.fit(X_train, y_train_categorical, validation_data=(X_test, y_test_categorical), epochs=10, batch_size=32)

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test, y_test_categorical)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Sauvegarder le modèle
model.save("face_recognition_model.h5")